In [69]:
# Train a model and make prediction on earnings
# Video-game dataset, y =Total earnings
# Data is scaled using min max scaler
# The mean squared error (MSE) for the test data set is: 0.0003009536094032228
# Earnings Prediction for Proposed Product - $260129.9568627

In [70]:
import pandas as pd
from keras.models import Sequential
from keras.layers import *
from sklearn.preprocessing import MinMaxScaler
import keras
import tensorflow as tf

In [71]:
# Load training data set from CSV file
training_data_df =pd.read_csv("sales_data_training.csv")

# Load testing data set from CSV file
test_data_df =pd.read_csv("sales_data_training.csv")

# Data needs to be scaled to a small range like 0 to 1 for the neural
# network to work well.
scaler =MinMaxScaler(feature_range=(0,1))

In [72]:
# Scale both the training inputs and outputs
training_data_df = scaler.fit_transform(training_data_df)
test_data_df = scaler.transform(test_data_df)

In [73]:
cols=["critic_rating","is_action","is_exclusive_to_us","is_portable","is_role_playing","is_sequel","is_sports","suitable_for_kids","total_earnings","unit_price"]

In [74]:
training_data_df=pd.DataFrame(training_data_df,columns=cols)
test_data_df=pd.DataFrame(test_data_df,columns=cols)

In [75]:
X = pd.DataFrame(training_data_df).drop('total_earnings', axis=1).values
Y = pd.DataFrame(training_data_df)[['total_earnings']].values

In [76]:
# Define the model
model = Sequential()
model.add(Dense(50, input_dim=9, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='linear'))
model.compile(loss='mean_squared_error', optimizer='adam')

In [77]:
# Create a TensorBoard logger

RUN_NAME = "run 2 with 5 nodes"

logger = keras.callbacks.TensorBoard(
    log_dir='logs/{}'.format(RUN_NAME),
#     histogram_freq=5,
#     write_graph=True
)

In [88]:
# Train the model
model.fit(
    X,
    Y,
    epochs=50,
    shuffle=True,
    verbose=0,
    callbacks=[logger]

)

In [79]:
X_test = test_data_df.drop('total_earnings', axis=1).values
Y_test = test_data_df[['total_earnings']].values

test_error_rate = model.evaluate(X_test, Y_test, verbose=0)
print("The mean squared error (MSE) for the test data set is: {}".format(test_error_rate))


The mean squared error (MSE) for the test data set is: 2.88548173266463e-05


In [89]:


model_builder = tf.saved_model.builder.SavedModelBuilder("exported_model-4")

inputs = {
    'input': tf.saved_model.utils.build_tensor_info(model.input)
}
outputs = {
    'earnings': tf.saved_model.utils.build_tensor_info(model.output)
}

signature_def = tf.saved_model.signature_def_utils.build_signature_def(
    inputs=inputs,
    outputs=outputs,
    method_name=tf.saved_model.signature_constants.PREDICT_METHOD_NAME
)

model_builder.add_meta_graph_and_variables(
    K.get_session(),
    tags=[tf.saved_model.tag_constants.SERVING],
    signature_def_map={
        tf.saved_model.signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY: signature_def
    }
)

model_builder.save()
# Train the model
model.fit(
    X,
    Y,
    epochs=50,
    shuffle=True,
    verbose=2,
    callbacks=[logger]
)

INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: b'exported_model-4/saved_model.pb'
Epoch 1/50
 - 0s - loss: 3.5297e-05
Epoch 2/50
 - 0s - loss: 2.9114e-05
Epoch 3/50
 - 0s - loss: 2.9368e-05
Epoch 4/50
 - 0s - loss: 3.0728e-05
Epoch 5/50
 - 0s - loss: 2.8416e-05
Epoch 6/50
 - 0s - loss: 2.3313e-05
Epoch 7/50
 - 0s - loss: 1.9854e-05
Epoch 8/50
 - 0s - loss: 1.7641e-05
Epoch 9/50
 - 0s - loss: 2.0547e-05
Epoch 10/50
 - 0s - loss: 4.2710e-05
Epoch 11/50
 - 0s - loss: 2.8156e-05
Epoch 12/50
 - 0s - loss: 1.1101e-04
Epoch 13/50
 - 0s - loss: 6.3853e-05
Epoch 14/50
 - 0s - loss: 3.4259e-05
Epoch 15/50
 - 0s - loss: 4.0078e-05
Epoch 16/50
 - 0s - loss: 2.5690e-05
Epoch 17/50
 - 0s - loss: 2.1629e-05
Epoch 18/50
 - 0s - loss: 2.5077e-05
Epoch 19/50
 - 0s - loss: 2.7972e-05
Epoch 20/50
 - 0s - loss: 3.2559e-05
Epoch 21/50
 - 0s - loss: 4.5247e-05
Epoch 22/50
 - 0s - loss: 2.6827e-05
Epoch 23/50
 - 0s - loss: 2.5690e-05
Epoch 24/50
 

In [84]:
model?

## Make a Single Prediction

In [ ]:
# Load the data we make to use to make a prediction
X = pd.read_csv("proposed_new_product.csv").values

# Make a prediction with the neural network
prediction = model.predict(X)

# Grab just the first element of the first prediction (since that's the only have one)
prediction = prediction[0][0]
# Re-scale the data from the 0-to-1 range back to dollars
# These constants are from when the data was originally scaled down to the 0-to-1 range
prediction = prediction + 0.1159
prediction = prediction / 0.0000036968

print("Earnings Prediction for Proposed Product - ${}".format(prediction))
